#### Ensemble - RandomForest & ExtraTree
- 배깅 방식의 앙상블 ==> 중복 랜덤 샘플 + 동일 모델 (DecisionTree)
    * 대표 알고리즘 : RandomForestC/R
- 페이스트 방식의 앙상블 ==> 랜덤 샘플 + 동일 모델 (DecisionTree)
    * 대표 알고리즘 : ExtraTreeC/R

[목표] 와인 분류 => 0과 1 2개의 종류 분류

[1] 모듈 로딩 및 데이터 준비

In [96]:
# 모듈 로딩 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [97]:
# 데이터
DATA_FILE = '../data/wine.csv'

# CSV => DataFrame
wineDF=pd.read_csv(DATA_FILE)

In [98]:
# 데이터 확인
wineDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [99]:
wineDF.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [100]:
# 타겟/라벨의 클래스 분포
wineDF['class'].value_counts()

class
1.0    4898
0.0    1599
Name: count, dtype: int64

In [101]:
wineDF.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


[2] 학습 준비

In [102]:
# 학습용 & 테스트용 데이터셋 분할
from sklearn.model_selection import train_test_split

In [103]:
featureDF=wineDF[wineDF.columns[:-1]]
targetSR= wineDF[wineDF.columns[-1]]

print(f'featureDF : {featureDF.shape}  targetSR : {targetSR.shape}')

featureDF : (6497, 3)  targetSR : (6497,)


In [104]:
# 학습용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test=train_test_split(featureDF, targetSR,test_size=0.2, stratify=targetSR, random_state=1)

In [105]:
print(f'X_train : {X_train.shape}  y_train : {y_train.shape}')
print(f'X_test : {X_test.shape}  y_test : {y_test.shape}')

X_train : (5197, 3)  y_train : (5197,)
X_test : (1300, 3)  y_test : (1300,)


[3] 학습 진행

In [106]:
# 학습 방법 : 지도학습 > 분류
# 알고리즘 : 앙상블 > 배깅 - RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [107]:
# 인스턴스 생성 => 100개의 내부 DT 모델에서 사용할 데이터셋 생성
#                 random_state 매개변수 설정으로 고정된 데이터셋 생성
#                 oob_score 매개변수 : 샘플 데이터셋 추출 후 남은 데이터셋 검증용으로 사용
lf_model=RandomForestClassifier(random_state=7, oob_score=True)

# 학습
lf_model.fit(X_train, y_train)

RandomForestClassifier(oob_score=True, random_state=7)

In [108]:
# 모델 파라미터
print(f'classes_ : {lf_model.classes_}')
print(f'n_classes_ : {lf_model.n_classes_}')
print(f'feature_names_in_ : {lf_model.feature_names_in_}')
print(f'n_features_in_ : {lf_model.n_features_in_}')
print(f'feature_importances_ : {lf_model.feature_importances_}')

classes_ : [0. 1.]
n_classes_ : 2
feature_names_in_ : ['alcohol' 'sugar' 'pH']
n_features_in_ : 3
feature_importances_ : [0.23572103 0.49995154 0.26432743]


In [109]:
# 모델 파라미터
print(f'classes_ : {lf_model.estimator_}')
for est in lf_model.estimators_:
    print(est)

classes_ : DecisionTreeClassifier()
DecisionTreeClassifier(max_features='sqrt', random_state=327741615)
DecisionTreeClassifier(max_features='sqrt', random_state=976413892)
DecisionTreeClassifier(max_features='sqrt', random_state=1202242073)
DecisionTreeClassifier(max_features='sqrt', random_state=1369975286)
DecisionTreeClassifier(max_features='sqrt', random_state=1882953283)
DecisionTreeClassifier(max_features='sqrt', random_state=2053951699)
DecisionTreeClassifier(max_features='sqrt', random_state=959775639)
DecisionTreeClassifier(max_features='sqrt', random_state=1956722279)
DecisionTreeClassifier(max_features='sqrt', random_state=2052949340)
DecisionTreeClassifier(max_features='sqrt', random_state=1322904761)
DecisionTreeClassifier(max_features='sqrt', random_state=165338510)
DecisionTreeClassifier(max_features='sqrt', random_state=1133316631)
DecisionTreeClassifier(max_features='sqrt', random_state=4812360)
DecisionTreeClassifier(max_features='sqrt', random_state=372560217)
Decisi

In [110]:
print(f'oob_score_ : {lf_model.oob_score_}')

oob_score_ : 0.89532422551472


[4] 성능평가

In [111]:
train_score=lf_model.score(X_train, y_train)
test_score=lf_model.score(X_test, y_test)

In [112]:
print(f'train_score : {train_score}  test_score : {test_score}')

train_score : 0.9973061381566288  test_score : 0.9


[5] 튜닝

- RandomizedSearchCV 하이퍼파라미터 최적화 클래스
    * 범위가 넓은 하이퍼파라미터 설정에 좋음
    * 지정된 범위에서 지정된 횟수만큼 하이퍼파라미터를 추출하여 조합 진행

In [113]:
from sklearn.model_selection import RandomizedSearchCV

In [114]:
# RandomForestClassifier 하이퍼파라미터 설정
params={'max_depth':range(2,16), 'min_samples_leaf':range(5,16), 'criterion':['gini', 'entropy', 'log_loss']}

In [115]:
rf_model=RandomForestClassifier(random_state=7)

In [116]:
searchCV=RandomizedSearchCV(rf_model, param_distributions=params,verbose=4, n_iter=50)
# verbose ==>진행 단계를 보고 싶을 때 사용

In [117]:
searchCV.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END criterion=entropy, max_depth=11, min_samples_leaf=14;, score=0.876 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=11, min_samples_leaf=14;, score=0.837 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=11, min_samples_leaf=14;, score=0.877 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=11, min_samples_leaf=14;, score=0.884 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=11, min_samples_leaf=14;, score=0.869 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, min_samples_leaf=8;, score=0.756 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, min_samples_leaf=8;, score=0.767 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, min_samples_leaf=8;, score=0.771 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, min_samples_leaf=8;, score=0.754 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, min_samples_leaf=8;, scor

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=7), n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': range(2, 16),
                                        'min_samples_leaf': range(5, 16)},
                   verbose=4)

In [118]:
# 모델 파라미터
print(f'[searchCV.best_score_] {searchCV.best_score_}')
print(f'[searchCV.best_params_] {searchCV.best_params_}')
print(f'[searchCV.best_estimator_] {searchCV.best_estimator_}')

cv_result=pd.DataFrame(searchCV.cv_results_)
cv_result

[searchCV.best_score_] 0.8760866587695268
[searchCV.best_params_] {'min_samples_leaf': 5, 'max_depth': 12, 'criterion': 'entropy'}
[searchCV.best_estimator_] RandomForestClassifier(criterion='entropy', max_depth=12, min_samples_leaf=5,
                       random_state=7)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.242617,0.012682,0.016604,0.001322,14,11,entropy,"{'min_samples_leaf': 14, 'max_depth': 11, 'cri...",0.875962,0.836538,0.876805,0.883542,0.869105,0.868390,0.016570,21
1,0.130266,0.006813,0.012590,0.006461,8,2,gini,"{'min_samples_leaf': 8, 'max_depth': 2, 'crite...",0.755769,0.767308,0.770934,0.753609,0.753609,0.760246,0.007379,47
2,0.275615,0.010026,0.014302,0.004496,7,13,gini,"{'min_samples_leaf': 7, 'max_depth': 13, 'crit...",0.876923,0.844231,0.879692,0.881617,0.873917,0.871276,0.013770,7
3,0.199961,0.010699,0.009861,0.008052,7,6,entropy,"{'min_samples_leaf': 7, 'max_depth': 6, 'crite...",0.856731,0.831731,0.870067,0.877767,0.863330,0.859925,0.015735,36
4,0.195189,0.005821,0.011489,0.005835,14,6,gini,"{'min_samples_leaf': 14, 'max_depth': 6, 'crit...",0.862500,0.830769,0.870067,0.879692,0.856593,0.859924,0.016501,37
5,0.278977,0.006328,0.014291,0.006897,6,12,log_loss,"{'min_samples_leaf': 6, 'max_depth': 12, 'crit...",0.880769,0.842308,0.877767,0.885467,0.876805,0.872623,0.015455,3
6,0.238471,0.009864,0.018928,0.004622,12,12,gini,"{'min_samples_leaf': 12, 'max_depth': 12, 'cri...",0.875962,0.834615,0.883542,0.886429,0.873917,0.870893,0.018720,10
7,0.266825,0.010600,0.012523,0.006402,8,13,log_loss,"{'min_samples_leaf': 8, 'max_depth': 13, 'crit...",0.875000,0.842308,0.877767,0.889317,0.873917,0.871662,0.015664,5
8,0.271016,0.006349,0.015579,0.001496,5,10,entropy,"{'min_samples_leaf': 5, 'max_depth': 10, 'crit...",0.874038,0.846154,0.880654,0.880654,0.874880,0.871276,0.012866,6
9,0.232871,0.006394,0.018467,0.004934,13,14,gini,"{'min_samples_leaf': 13, 'max_depth': 14, 'cri...",0.874038,0.833654,0.873917,0.878730,0.870067,0.866081,0.016445,30
